In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

DDoS = pd.read_csv('/content/SDN_DDoS_.csv')
DDoS.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,245230,44,40,124937,1071,9100,0,2839.477273,1839.508257,517,0,26.775000,109.188026,513835.99070,342.535579,2954.578313,7953.221927,64066,-44,238564,5548.000000,10446.29576,64066,2,245230,6287.948718,12986.46879,79070,29,0,0,880,804,179.423398,163.112180,0,9100,1482.447059,1933.268313,3.737526e+06,0,1,0,0,1,0,0,0,0,1500.095238,2839.477273,26.775000,44,124937,40,1071,41,0,0.0,0.0,0,0,0.0,0.0,0,0,0
1,1605449,107,149,1071,439537,517,0,10.009346,67.496680,27300,0,2949.912752,3012.589539,274445.34210,159.456949,6295.878431,56408.330520,859760,-102,1332121,12567.179250,83434.14155,861138,2,1603130,10831.959460,73926.65245,861129,1,0,0,2140,3004,66.648022,92.808928,0,27300,1714.428016,2713.465917,7.362897e+06,0,1,0,0,0,0,0,0,1,1721.125000,10.009346,2949.912752,107,1071,149,439537,4,0,0.0,0.0,0,0,0.0,0.0,0,0,0
2,53078,5,5,66,758,66,0,13.200000,29.516097,638,0,151.600000,276.826299,15524.32269,188.401975,5897.555556,15184.845200,46232,19,50302,12575.500000,22521.87727,46251,67,52962,13240.500000,22052.04405,46258,405,0,0,100,124,94.200987,94.200987,0,638,74.909091,190.807471,3.640749e+04,0,1,0,0,0,0,0,0,1,82.400000,13.200000,151.600000,5,66,5,758,1,0,0.0,0.0,0,0,0.0,0.0,0,0,0
3,6975,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.00000,286.738351,6975.000000,0.000000,6975,6975,0,0.000000,0.00000,0,0,0,0.000000,0.00000,0,0,0,0,20,20,143.369176,143.369176,0,0,0.000000,0.000000,0.000000e+00,0,0,0,0,1,0,0,0,1,0.000000,0.000000,0.000000,1,0,1,0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0
4,190141,13,16,780,11085,427,0,60.000000,130.042942,2596,0,692.812500,794.157350,62401.06027,152.518394,6790.750000,12933.295910,38521,-54,86882,7240.166667,13050.84163,38805,1,190141,12676.066670,15949.09279,38521,1,0,0,260,344,68.370315,84.148080,0,2596,395.500000,661.691706,4.378359e+05,0,1,0,0,0,0,0,0,1,409.137931,60.000000,692.812500,13,780,16,11085,3,0,0.0,0.0,0,0,0.0,0.0,0,0,0


In [ ]:
DDoS.isnull().sum()

Flow Duration      0
Tot Fwd Pkts       0
Tot Bwd Pkts       0
TotLen Fwd Pkts    0
TotLen Bwd Pkts    0
                  ..
Idle Mean          0
Idle Std           0
Idle Max           0
Idle Min           0
Label              0
Length: 67, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
features = DDoS.drop('Label', axis=1)
labels = DDoS['Label']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [ ]:
for dataset in [y_train, y_val, y_test]:
    print(round(len(dataset) / len(labels), 2))

0.6
0.2
0.2


In [ ]:
X_train.to_csv('train_features.csv', index=False)
X_val.to_csv('val_features.csv', index=False)
X_test.to_csv('test_features.csv', index=False)

y_train.to_csv('train_labels.csv', index=False)
y_val.to_csv('val_labels.csv', index=False)
y_test.to_csv('test_labels.csv', index=False)

In [ ]:
import joblib
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

tr_features = pd.read_csv('train_features.csv')
tr_labels = pd.read_csv('train_labels.csv')
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

lr = LogisticRegression()
parameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

cv = GridSearchCV(lr, parameters, cv=5)
cv.fit(tr_features, tr_labels.values.ravel())

print_results(cv)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


BEST PARAMS: {'C': 1000}

1.0 (+/-0.001) for {'C': 0.001}
0.999 (+/-0.002) for {'C': 0.01}
1.0 (+/-0.001) for {'C': 0.1}
1.0 (+/-0.0) for {'C': 1}
1.0 (+/-0.001) for {'C': 10}
1.0 (+/-0.0) for {'C': 100}
1.0 (+/-0.0) for {'C': 1000}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
cv.best_estimator_

LogisticRegression(C=1000)

In [ ]:
joblib.dump(cv.best_estimator_, 'LR_model.pkl')

['LR_model.pkl']

In [ ]:
from sklearn.svm import SVC
svc = SVC()
parameters = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10]
}

cv = GridSearchCV(svc, parameters, cv=5)
cv.fit(tr_features, tr_labels.values.ravel())

print_results(cv)

BEST PARAMS: {'C': 0.1, 'kernel': 'linear'}

1.0 (+/-0.0) for {'C': 0.1, 'kernel': 'linear'}
0.9 (+/-0.0) for {'C': 0.1, 'kernel': 'rbf'}
1.0 (+/-0.0) for {'C': 1, 'kernel': 'linear'}
0.9 (+/-0.0) for {'C': 1, 'kernel': 'rbf'}
1.0 (+/-0.0) for {'C': 10, 'kernel': 'linear'}
0.9 (+/-0.0) for {'C': 10, 'kernel': 'rbf'}


In [ ]:
cv.best_estimator_

SVC(C=0.1, kernel='linear')

In [ ]:
joblib.dump(cv.best_estimator_, 'SVM_model.pkl')

['SVM_model.pkl']

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
parameters = {
    'hidden_layer_sizes': [(10,), (50,), (100,)],
    'activation': ['relu', 'tanh', 'logistic'],
    'learning_rate': ['constant', 'invscaling', 'adaptive']
}

cv = GridSearchCV(mlp, parameters, cv=5)
cv.fit(tr_features, tr_labels.values.ravel())

print_results(cv)

BEST PARAMS: {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'invscaling'}

0.999 (+/-0.001) for {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'constant'}
0.998 (+/-0.003) for {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'invscaling'}
0.999 (+/-0.003) for {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'adaptive'}
0.999 (+/-0.001) for {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'constant'}
0.999 (+/-0.0) for {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'invscaling'}
0.999 (+/-0.001) for {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'adaptive'}
0.999 (+/-0.0) for {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'constant'}
0.999 (+/-0.001) for {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'invscaling'}
0.997 (+/-0.006) for {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning

In [ ]:
cv.best_estimator_

MLPClassifier(hidden_layer_sizes=(50,), learning_rate='invscaling')

In [ ]:
joblib.dump(cv.best_estimator_, 'MLP_model.pkl')

['MLP_model.pkl']

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
parameters = {
    'n_estimators': [5, 50, 250],
    'max_depth': [2, 4, 8, 16, 32, None]
}

cv = GridSearchCV(rf, parameters, cv=5)
cv.fit(tr_features, tr_labels.values.ravel())

print_results(cv)

BEST PARAMS: {'max_depth': 8, 'n_estimators': 50}

0.994 (+/-0.004) for {'max_depth': 2, 'n_estimators': 5}
0.997 (+/-0.003) for {'max_depth': 2, 'n_estimators': 50}
0.998 (+/-0.002) for {'max_depth': 2, 'n_estimators': 250}
0.999 (+/-0.0) for {'max_depth': 4, 'n_estimators': 5}
1.0 (+/-0.0) for {'max_depth': 4, 'n_estimators': 50}
1.0 (+/-0.0) for {'max_depth': 4, 'n_estimators': 250}
1.0 (+/-0.0) for {'max_depth': 8, 'n_estimators': 5}
1.0 (+/-0.0) for {'max_depth': 8, 'n_estimators': 50}
1.0 (+/-0.0) for {'max_depth': 8, 'n_estimators': 250}
1.0 (+/-0.0) for {'max_depth': 16, 'n_estimators': 5}
1.0 (+/-0.0) for {'max_depth': 16, 'n_estimators': 50}
1.0 (+/-0.0) for {'max_depth': 16, 'n_estimators': 250}
1.0 (+/-0.0) for {'max_depth': 32, 'n_estimators': 5}
1.0 (+/-0.0) for {'max_depth': 32, 'n_estimators': 50}
1.0 (+/-0.0) for {'max_depth': 32, 'n_estimators': 250}
1.0 (+/-0.0) for {'max_depth': None, 'n_estimators': 5}
1.0 (+/-0.0) for {'max_depth': None, 'n_estimators': 50}
1.0 (+

In [ ]:
cv.best_estimator_

RandomForestClassifier(max_depth=8, n_estimators=50)

In [ ]:
joblib.dump(cv.best_estimator_, 'RF_model.pkl')

['RF_model.pkl']

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
parameters = {
    'n_estimators': [5, 50, 250, 500],
    'max_depth': [1, 3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 1, 10, 100]
}

cv = GridSearchCV(gb, parameters, cv=5)
cv.fit(tr_features, tr_labels.values.ravel())

print_results(cv)

BEST PARAMS: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 250}

0.9 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 5}
0.9 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 50}
1.0 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 250}
1.0 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500}
0.9 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 5}
0.9 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}
1.0 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 250}
1.0 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
0.9 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 5}
0.9 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50}
1.0 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 250}
1.0 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 500

In [19]:
cv.best_estimator_

GradientBoostingClassifier(learning_rate=0.01, max_depth=1, n_estimators=250)

In [20]:
joblib.dump(cv.best_estimator_, 'GB_model.pkl')

['GB_model.pkl']

In [25]:
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score
from time import time

val_features = pd.read_csv('val_features.csv')
val_labels = pd.read_csv('val_labels.csv')

te_features = pd.read_csv('test_features.csv')
te_labels = pd.read_csv('test_labels.csv')

In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
models = {}

for mdl in ['LR', 'SVM', 'MLP', 'RF', 'GB']:
    models[mdl] = joblib.load('{}_model.pkl'.format(mdl))
    

In [27]:
def evaluate_model(name, model, features, labels):
    start = time()
    pred = model.predict(features)
    end = time()
    accuracy = round(accuracy_score(labels, pred), 3)
    precision = round(precision_score(labels, pred), 3)
    recall = round(recall_score(labels, pred), 3)
    print('{} -- Accuracy: {} / Precision: {} / Recall: {} / Latency: {}ms'.format(name,
                                                                                   accuracy,
                                                                                   precision,
                                                                                   recall,
                                                                                   round((end - start)*1000, 1)))

In [28]:
for name, mdl in models.items():
    evaluate_model(name, mdl, val_features, val_labels)

LR -- Accuracy: 1.0 / Precision: 1.0 / Recall: 1.0 / Latency: 24.4ms
SVM -- Accuracy: 1.0 / Precision: 0.999 / Recall: 1.0 / Latency: 30.9ms
MLP -- Accuracy: 1.0 / Precision: 0.999 / Recall: 0.999 / Latency: 31.5ms
RF -- Accuracy: 1.0 / Precision: 1.0 / Recall: 1.0 / Latency: 71.3ms
GB -- Accuracy: 1.0 / Precision: 1.0 / Recall: 1.0 / Latency: 52.9ms


In [29]:
evaluate_model('Random Forest', models['RF'], te_features, te_labels)

Random Forest -- Accuracy: 1.0 / Precision: 1.0 / Recall: 0.999 / Latency: 56.4ms
